Everything to Get Current Season

Scrape

In [7]:
import pandas as pd
import csv
import os
from pathlib import Path

# Set working directory to project root if run from notebooks/
if Path.cwd().name == "notebooks":
    os.chdir("..")

# Define folders – assume they already exist
raw_data_dir = Path("data/Raw")
cleaned_data_dir = Path("data/Cleaned")
metrics_data_dir = Path("data/Metrics")
current_season_dir = metrics_data_dir / "CurrentSeasonTeams"

url = 'https://www.basketball-reference.com/wnba/years/2025_totals.html'
df2025 = pd.read_html(url)[0]  # First table is the player totals
df2025.to_csv(raw_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')
df2025 = df2025[df2025['Player'] != 'Player']  # Drop header rows repeated in table
df2025 = df2025.drop_duplicates()        # Remove duplicate entries, if any
tot_players = df2025[df2025['Team'] == 'TOT']['Player']
df2025 = pd.concat([
    df2025[df2025['Team'] == 'TOT'],                              # all TOT rows
    df2025[~df2025['Player'].isin(tot_players) & (df2025['Team'] != 'TOT')]  # single-team players
], ignore_index=True)
df2025['MP'] = pd.to_numeric(df2025['MP'])
df2025 = df2025.drop('G.1', axis=1)
df2025 = df2025.drop('MP.1', axis=1)

cols_to_exclude = ['Player', 'Team', 'Pos']
cols_to_convert = df2025.columns.difference(cols_to_exclude)

df2025[cols_to_convert] = df2025[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df2025

,Player,Team,Pos,G,MP,GS,FG,FGA,FG%,3P,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Grace Berger,TOT,G,13,253,7,15,59,0.254,1,...,11,0.545,6,37,34,9,5,16,28,37
1,Chloe Bibby,TOT,F,14,154,0,21,55,0.382,17,...,9,0.889,5,25,7,8,1,7,13,67
2,DeWanna Bonner,TOT,G-F,26,605,4,82,199,0.412,31,...,71,0.873,21,107,30,18,6,22,34,257
3,DiJonai Carrington,TOT,G-F,28,640,13,97,258,0.376,20,...,76,0.803,47,120,45,29,12,58,72,275
4,Kaila Charles,TOT,G-F,28,434,5,50,125,0.400,19,...,41,0.829,40,102,25,18,5,26,51,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Gabby Williams,SEA,F,38,1224,38,180,415,0.434,45,...,69,0.841,22,167,167,90,19,73,53,463
174,Kiana Williams,PHO,G,8,76,0,12,25,0.480,5,...,2,0.500,0,7,7,1,0,2,2,30
175,A'ja Wilson,LVA,C,34,1052,34,276,559,0.494,15,...,260,0.854,80,346,106,54,74,78,69,789
176,Jackie Young,LVA,G,38,1146,38,218,482,0.452,62,...,138,0.906,48,167,176,49,14,89,99,623


Adding DRB

In [8]:
df2025['DRB'] = df2025['TRB'] - df2025['ORB']

# Define target and reference columns
col_to_move = 'DRB'
before_col = 'TRB'

# Get current column order and remove the column to move
cols = df2025.columns.tolist()
cols.remove(col_to_move)

# Find index to insert the column before the target
insert_at = cols.index(before_col)

# Insert the column at the desired position
cols.insert(insert_at, col_to_move)

# Reorder the DataFrame
df2025 = df2025[cols]

df2025.to_csv(cleaned_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Metrics

In [9]:
df2025 = pd.read_csv(cleaned_data_dir / '2025Data.csv')

df2025['Efficiency'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['AST'] + df2025['STL'] - df2025['TOV'] - (df2025['FGA'] - df2025['FG']) - (df2025['FTA'] - df2025['FT']))

df2025['WinScore'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['STL'] + (df2025['AST'] * 0.5) + (df2025['BLK'] * 0.5) - df2025['FGA'] - df2025['TOV'] - (df2025['FTA'] * 0.5) - (df2025['PF'] * 0.5))

df2025['NBAPER'] = .68 + ((1/df2025['MP']) * ((26.77 * df2025['FG']) - (35.58 * df2025['FGA']) + (26.32 * df2025['FT']) - (25.47 * df2025['FTA']) + (42.74 * df2025['ORB']) + (11.45 * df2025['DRB']) + (29.7 * df2025['AST']) + (47.3 * df2025['STL']) + (38 * df2025['BLK']) - (47.9 * df2025['TOV']) - (18.8 * df2025['PF']) + (41.1 * df2025['PTS'])))

df2025['WNBAPER'] = .3289 + ((1/df2025['MP']) * ((53.35 * df2025['FG']) - (29.28 * df2025['FGA']) + (15.27 * df2025['FT']) - (25.34 * df2025['FTA']) + (47.72 * df2025['ORB']) + (10.95 * df2025['DRB']) + (31.61 * df2025['AST']) + (50.72 * df2025['STL']) + (50.68 * df2025['BLK']) - (45.88 * df2025['TOV']) - (61.23 * df2025['PF']) + (41.1 * df2025['PTS'])))

print("Columns before filtering (2025):", df2025.columns.tolist())

expected_cols = ['Player', 'Team', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']

df2025 = df2025[[col for col in expected_cols if col in df2025.columns]]

df2025.to_csv(current_season_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Columns before filtering (2025): ['Player', 'Team', 'Pos', 'G', 'MP', 'GS', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Efficiency', 'WinScore', 'NBAPER', 'WNBAPER']


Teamless

In [10]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025teamless = df2025.drop(columns=['Team'])

df2025teamless = df2025teamless.round(3)

df2025teamless.to_csv(metrics_data_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Team Specific

In [11]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025 = df2025.round(3)

# The column to split on
split_column = "Team"

# Directory to save files (ensure it exists or create it)
output_dir = current_season_dir

# Split and save
for group, group_df in df2025.groupby(split_column):
    filename = f"{output_dir}/{group}.csv"
    group_df = group_df.drop(columns=['Team'])  # Drop 'Team' column
    group_df.to_csv(filename, index=False, sep=',', encoding='utf-8', lineterminator='\n')

Combined DFs

In [12]:

df2023 = pd.read_csv(metrics_data_dir / '2023Metrics.csv')

df2024 = pd.read_csv(metrics_data_dir / '2024Metrics.csv')

df2025 = pd.read_csv(metrics_data_dir / '2025Metrics.csv')

combined = pd.concat([
    df2023.assign(Season=2023),
    df2024.assign(Season=2024),
    df2025.assign(Season=2025)
])

combined[['Player', 'Season', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']].to_csv(metrics_data_dir / 'AllPlayersMetrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')